In [1]:
import os
import wandb
# print('here!!!!',os.getcwd())
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wangxinyan (wxy-ucas). Use `wandb login --relogin` to force relogin


True

In [2]:
import time
import os
import argparse
import numpy as np
import torch
from torch_geometric.utils import add_remaining_self_loops, to_undirected

from GNN import GNN
from GNN_early import GNNEarly
from data import get_dataset, set_train_val_test_split
from heterophilic import get_fixed_splits
from graph_rewiring import apply_beltrami

from gread_params import best_params_dict, hetero_params, shared_gread_params, shared_grand_params
from utils import dirichlet_energy
import wandb

/root/miniconda3/envs/gread/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [3]:
parser = argparse.ArgumentParser()
#run args
parser.add_argument('--use_best_params', action='store_true', help="flag to take the best params of GREAD")
parser.add_argument('--gpu', type=int, default=0, help="GPU to run on (default 0)")
parser.add_argument('--epoch', type=int, default=100, help='Number of training epochs per iteration.')
parser.add_argument('--patience', type=int, default=None, help='set if training should use patience on val acc')
parser.add_argument('--optimizer', type=str, default='adam', help='One from sgd, rmsprop, adam, adagrad, adamax.')
parser.add_argument('--lr', type=float, default=0.01, help='Learning rate.')
parser.add_argument('--decay', type=float, default=5e-4, help='Weight decay for optimization')

# data args
parser.add_argument('--dataset', type=str, default='Cora', help='Cora, Citeseer, Pubmed, Computers, Photo, CoauthorCS, ogbn-arxiv')
parser.add_argument('--data_norm', type=str, default='rw', help='rw for random walk, gcn for symmetric gcn norm')
parser.add_argument('--self_loop_weight', type=float, default=0.0,help='Weight of self-loops.')
parser.add_argument('--use_labels', dest='use_labels', action='store_true', help='Also diffuse labels')
parser.add_argument('--label_rate', type=float, default=0.5, help='% of training labels to use when --use_labels is set.')
parser.add_argument('--planetoid_split', action='store_true', help='use planetoid splits for Cora/Citeseer/Pubmed')
parser.add_argument('--geom_gcn_splits', dest='geom_gcn_splits', action='store_true', help='use the 10 fixed splits from https://arxiv.org/abs/2002.05287')
parser.add_argument('--num_splits', type=int, dest='num_splits', default=1, help='the number of splits to repeat the results on')
parser.add_argument("--not_lcc", action="store_false", help="don't use the largest connected component")
parser.add_argument('--hetero_SL', action='store_true', help='control self loops for Chameleon/Squirrel')
parser.add_argument('--hetero_undir', action='store_true', help='control undirected for Chameleon/Squirrel')

# GNN args
parser.add_argument('--block', type=str, default='constant', help='constant, mixed, attention, hard_attention')
parser.add_argument('--function', type=str, default='gread', help='laplacian, transformer, gread, GAT') # let function is laplacian then it is the grand model
parser.add_argument('--hidden_dim', type=int, default=64, help='Hidden dimension.')
parser.add_argument('--fc_out', type=eval, default=False, help='Add a fully connected layer to the decoder.')
parser.add_argument('--input_dropout', type=float, default=0.5, help='Input dropout rate.')
parser.add_argument('--dropout', type=float, default=0.0, help='Dropout rate.')
parser.add_argument("--batch_norm", dest='batch_norm', action='store_true', help='search over reg params')
parser.add_argument('--alpha', type=float, default=1.0, help='Factor in front matrix A.')
parser.add_argument('--alpha_dim', type=str, default='sc', help='choose either scalar (sc) or vector (vc) alpha')
parser.add_argument('--no_alpha_sigmoid', dest='no_alpha_sigmoid', action='store_true', help='apply sigmoid before multiplying by alpha')
parser.add_argument('--source_dim', type=str, default='sc', help='choose either scalar (sc) or vector (vc) source')
parser.add_argument('--beta_dim', type=str, default='sc', help='choose either scalar (sc) or vector (vc) beta')
parser.add_argument('--use_mlp', type=eval, default=False, help='Add a fully connected layer to the encoder.')
parser.add_argument('--add_source', type=eval, default=False, help='If try get rid of alpha param and the source*x0 source term')
parser.add_argument('--XN_activation', type=eval, default=False, help='whether to relu activate the terminal state')
parser.add_argument('--m2_mlp', type=eval, default=False, help='whether to use decoder mlp')

# ODE args
parser.add_argument('--time', type=float, default=1.0, help='End time of ODE integrator.')
parser.add_argument('--augment', type=eval, default=False, help='double the length of the feature vector by appending zeros to stabilist ODE learning')
parser.add_argument('--method', type=str, help="set the numerical solver: dopri5, euler, rk4, midpoint, symplectic_euler, leapfrog")
parser.add_argument('--step_size', type=float, default=0.1, help='fixed step size when using fixed step solvers e.g. rk4')
parser.add_argument('--max_iters', type=float, default=100, help='maximum number of integration steps')
parser.add_argument("--adjoint_method", type=str, default="adaptive_heun", help="set the numerical solver for the backward pass: dopri5, euler, rk4, midpoint")
parser.add_argument('--adjoint', dest='adjoint', action='store_true', help='use the adjoint ODE method to reduce memory footprint')
parser.add_argument('--adjoint_step_size', type=float, default=1, help='fixed step size when using fixed step adjoint solvers e.g. rk4')
parser.add_argument('--tol_scale', type=float, default=1., help='multiplier for atol and rtol')
parser.add_argument("--tol_scale_adjoint", type=float, default=1.0, help="multiplier for adjoint_atol and adjoint_rtol")
# 适当增加max_nfe: 原数值：1000
parser.add_argument("--max_nfe", type=int, default=5000, help="Maximum number of function evaluations in an epoch. Stiff ODEs will hang if not set.")
parser.add_argument('--no_early', type=eval, default=True)
parser.add_argument('--earlystopxT', type=float, default=3, help='multiplier for T used to evaluate best model')
parser.add_argument("--max_test_steps", type=int, default=100, help="Maximum number steps for the dopri5Early test integrator. used if getting OOM errors at test time")

# regularisation args
parser.add_argument('--jacobian_norm2', type=float, default=None, help="int_t ||df/dx||_F^2")
parser.add_argument('--total_deriv', type=float, default=None, help="int_t ||df/dt||^2")
parser.add_argument('--kinetic_energy', type=float, default=None, help="int_t ||f||_2^2")
parser.add_argument('--directional_penalty', type=float, default=None, help="int_t ||(df/dx)^T f||^2")
# Attention args
# 默认单头注意力
parser.add_argument('--leaky_relu_slope', type=float, default=0.2,
                    help='slope of the negative part of the leaky relu used in attention')
parser.add_argument('--attention_dropout', type=float, default=0., help='dropout of attention weights')
parser.add_argument('--heads', type=int, default=1, help='number of attention heads')
parser.add_argument('--attention_norm_idx', type=int, default=0, help='0 = normalise rows, 1 = normalise cols')
parser.add_argument('--attention_dim', type=int, default=64,
                    help='the size to project x to before calculating att scores')
parser.add_argument('--mix_features', dest='mix_features', action='store_true',
                    help='apply a feature transformation xW to the ODE')
parser.add_argument('--reweight_attention', dest='reweight_attention', action='store_true',
                    help="multiply attention scores by edge weights before softmax")
parser.add_argument('--attention_type', type=str, default="scaled_dot",
                    help="scaled_dot,cosine_sim,pearson, exp_kernel")
parser.add_argument('--square_plus', action='store_true', help='replace softmax with square plus')

# GCN ablation args
parser.add_argument('--gcn_fixed', type=str, default='False', help='fixes layers in gcn')
parser.add_argument('--gcn_enc_dec', type=str, default='False', help='uses encoder decoder with GCN')
parser.add_argument('--gcn_non_lin', type=str, default='False', help='uses non linearity with GCN')
parser.add_argument('--gcn_symm', type=str, default='False', help='make weight matrix in GCN symmetric')
parser.add_argument('--gcn_bias', type=str, default='False', help='make GCN include bias')
parser.add_argument('--gcn_mid_dropout', type=str, default='False', help='dropout between GCN layers')
parser.add_argument('--gcn_params', nargs='+', default=None, help='list of args for gcn ablation')
parser.add_argument('--gcn_params_idx', type=int, default=0, help='index to track GCN ablation')

# rewiring args
parser.add_argument('--rewiring', type=str, default=None, help="two_hop, gdc")
parser.add_argument('--gdc_method', type=str, default='ppr', help="ppr, heat, coeff")
parser.add_argument('--gdc_sparsification', type=str, default='topk', help="threshold, topk")
parser.add_argument('--gdc_k', type=int, default=64, help="number of neighbours to sparsify to when using topk")
parser.add_argument('--gdc_threshold', type=float, default=0.0001,
                    help="obove this edge weight, keep edges when using threshold")
parser.add_argument('--gdc_avg_degree', type=int, default=64,
                    help="if gdc_threshold is not given can be calculated by specifying avg degree")
parser.add_argument('--ppr_alpha', type=float, default=0.05, help="teleport probability")
parser.add_argument('--heat_time', type=float, default=3., help="time to run gdc heat kernal diffusion for")
parser.add_argument('--att_samp_pct', type=float, default=1,
                    help="float in [0,1). The percentage of edges to retain based on attention scores")
parser.add_argument('--use_flux', dest='use_flux', action='store_true',
                    help='incorporate the feature grad in attention based edge dropout')
parser.add_argument("--exact", action="store_true",
                    help="for small datasets can do exact diffusion. If dataset is too big for matrix inversion then you can't")
parser.add_argument('--M_nodes', type=int, default=64, help="new number of nodes to add")
parser.add_argument('--new_edges', type=str, default="random", help="random, random_walk, k_hop")
parser.add_argument('--sparsify', type=str, default="S_hat", help="S_hat, recalc_att")
parser.add_argument('--threshold_type', type=str, default="topk_adj", help="topk_adj, addD_rvR")
parser.add_argument('--rw_addD', type=float, default=0.02, help="percentage of new edges to add")
parser.add_argument('--rw_rmvR', type=float, default=0.02, help="percentage of edges to remove")
parser.add_argument('--rewire_KNN', action='store_true', help='perform KNN rewiring every few epochs')
parser.add_argument('--rewire_KNN_T', type=str, default="T0", help="T0, TN")
parser.add_argument('--rewire_KNN_epoch', type=int, default=5, help="frequency of epochs to rewire")
parser.add_argument('--rewire_KNN_k', type=int, default=64, help="target degree for KNN rewire")
parser.add_argument('--rewire_KNN_sym', action='store_true', help='make KNN symmetric')
parser.add_argument('--KNN_online', action='store_true', help='perform rewiring online')
parser.add_argument('--KNN_online_reps', type=int, default=4, help="how many online KNN its")
parser.add_argument('--KNN_space', type=str, default="pos_distance", help="Z,P,QKZ,QKp")
parser.add_argument('--pos_enc_csv', action='store_true', help="Generate pos encoding as a sparse CSV")

# beltrami args
parser.add_argument('--beltrami', type=eval, default=False, help='perform diffusion beltrami style')
parser.add_argument('--fa_layer', type=eval, default=False, help='add a bottleneck paper style layer with more edges')
parser.add_argument('--pos_enc_type', type=str, default="GDC",
                    help='positional encoder either GDC, DW64, DW128, DW256')
parser.add_argument('--pos_enc_orientation', type=str, default="row", help="row, col")
parser.add_argument('--feat_hidden_dim', type=int, default=64, help="dimension of features in beltrami")
parser.add_argument('--pos_enc_hidden_dim', type=int, default=32, help="dimension of position in beltrami")
parser.add_argument('--edge_sampling', action='store_true', help='perform edge sampling rewiring')
parser.add_argument('--edge_sampling_T', type=str, default="T0", help="T0, TN")
parser.add_argument('--edge_sampling_epoch', type=int, default=5, help="frequency of epochs to rewire")
parser.add_argument('--edge_sampling_add', type=float, default=0.64, help="percentage of new edges to add")
parser.add_argument('--edge_sampling_add_type', type=str, default="importance",
                    help="random, ,anchored, importance, degree")
parser.add_argument('--edge_sampling_rmv', type=float, default=0.32, help="percentage of edges to remove")
parser.add_argument('--edge_sampling_sym', action='store_true', help='make KNN symmetric')
parser.add_argument('--edge_sampling_online', action='store_true', help='perform rewiring online')
parser.add_argument('--edge_sampling_online_reps', type=int, default=4, help="how many online KNN its")
parser.add_argument('--edge_sampling_space', type=str, default="attention",
                    help="attention,pos_distance, z_distance, pos_distance_QK, z_distance_QK")

# gread args
parser.add_argument('--reaction_term', type=str, default='aggdiff-gat', help='bspm, fisher, allen-cahn')
parser.add_argument('--beta_diag', type=eval, default=False)

# with source term args
parser.add_argument('--trusted_mask', type=eval, default=False, help='mask')
parser.add_argument('--noise', type=float, default=0.0)
parser.add_argument('--noise_pos', type=str, help='all, test')
parser.add_argument('--prediffuse', action='store_true')
parser.add_argument('--x0', action='store_true')
parser.add_argument('--nox0', type=eval, default=False)
parser.add_argument('--icxb', type=float, default=1.0)
parser.add_argument('--source_scale', type=float, default=1.0)
parser.add_argument('--alltime', action='store_true')
parser.add_argument('--allnumtrain', action='store_true')
parser.add_argument("--num_train_per_class", type=int, default=20)

# visualize-related args
parser.add_argument('--save_result', type=eval, default=False, help='')

parser.add_argument('--wandb', action='store_true', help="flag if logging to wandb")
parser.add_argument('--wandb_sweep', action='store_true',help="flag if sweeping")
parser.add_argument('--wandb_entity', default="username", type=str)
parser.add_argument('--wandb_project', default="grade", type=str)
parser.add_argument('--wandb_run_name', default=None, type=str)
parser.add_argument('--run_track_reports', action='store_true', help="run_track_reports")
parser.add_argument('--save_wandb_reports', action='store_true', help="save_wandb_reports")
args = parser.parse_args()
opt = vars(args)

usage: ipykernel_launcher.py [-h] [--use_best_params] [--gpu GPU]
                             [--epoch EPOCH] [--patience PATIENCE]
                             [--optimizer OPTIMIZER] [--lr LR] [--decay DECAY]
                             [--dataset DATASET] [--data_norm DATA_NORM]
                             [--self_loop_weight SELF_LOOP_WEIGHT]
                             [--use_labels] [--label_rate LABEL_RATE]
                             [--planetoid_split] [--geom_gcn_splits]
                             [--num_splits NUM_SPLITS] [--not_lcc]
                             [--hetero_SL] [--hetero_undir] [--block BLOCK]
                             [--function FUNCTION] [--hidden_dim HIDDEN_DIM]
                             [--fc_out FC_OUT] [--input_dropout INPUT_DROPOUT]
                             [--dropout DROPOUT] [--batch_norm]
                             [--alpha ALPHA] [--alpha_dim ALPHA_DIM]
                             [--no_alpha_sigmoid] [--source_dim SOURCE_DIM]
 

SystemExit: 2

/root/miniconda3/envs/gread/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
